<a href="https://colab.research.google.com/github/Helenessli/xraytoolingpersonal/blob/master/DenseNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Datasets/dataset.csv')
df.head()

,image_id,hand,leg,hip,shoulder,mixed,hardware,multiscan,fractured,fracture_count,frontal,lateral,oblique
0,IMG0000000.jpg,0,1,0,0,0,0,1,0,0,1,1,0
1,IMG0000001.jpg,0,1,0,0,0,0,1,0,0,1,1,0
2,IMG0000002.jpg,0,1,0,0,0,0,1,0,0,1,1,0
3,IMG0000003.jpg,0,1,0,0,0,0,1,0,0,0,1,1
4,IMG0000004.jpg,0,1,0,0,0,0,1,0,0,0,1,1


In [3]:
# Copy the dataset to Colab
!cp -r '/content/drive/MyDrive/Datasets/dataset' '/content/'


In [4]:
# basic imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torchvision.models.densenet import DenseNet121_Weights

In [5]:
# cuda availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# transform to imagenet format
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
# Load the dataset
train_dataset = datasets.ImageFolder("dataset/train", transform=transform)
val_dataset = datasets.ImageFolder("dataset/val", transform=transform)

# Adjust labels to binary: fractured=1, not_fractured=0
# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)


In [7]:
# Pretrained, densenet model for imagenet
model = models.densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 82.2MB/s]


In [8]:
# freezing all layers in the model
for param in model.parameters():
    param.requires_grad = False

In [9]:
# Replacing number of classes to classify into, to our desired number
num_classes = 2 # binary classification
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)  # binary classification, so 2 output neuron
model = model.to(device)
#model = nn.Sequential(model, nn.Sigmoid())  #use Sigmoid as the final activation, commonly used for binary classification

In [10]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Define scheduler (StepLR)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [11]:
correct = 0
total = 0
#torch.set_grad_enabled(True)
# play around with this number
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    # Validation Accuracy
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = torch.round(model(inputs))  # convert probabilities to binary predictions

            # loop through each prediction and label to check accuracy
            for output, label in zip(outputs, labels):
                #normalize the vector and then round it to get a scalar
                result = round(sum(output.numpy())/1000)

                predicted_class = "fractured" if result == 0 else "not fractured"
                true_class = "fractured" if label.item() == 1 else "not fractured"

                # Check accuracy
                total += 1
                if predicted_class == true_class:
                    correct += 1

            # Calculate accuracy
    accuracy = correct / total * 100
    print(f'Accuracy after epoch {epoch + 1}: {accuracy:.2f}%')


# Decay learning rate using scheduler
scheduler.step()

Accuracy after epoch 1: 74.58%
Accuracy after epoch 2: 74.58%
Accuracy after epoch 3: 74.58%
Accuracy after epoch 4: 74.58%
Accuracy after epoch 5: 74.58%
